## 1. Presentació del conjunt de dades escollit
Dades que recullen els **accidents de tràfic a la ciutat de Barcelona**.

Disposem de dades des de l’any 2010 fins a 2023 (dades mes recents) repartides en 6 conjunts de dades anuals disponibles a [Open Data BCN](https://opendata-ajuntament.barcelona.cat/ca/).

Començem analitzant les dades de 2023, per després agregar les dades dels últims deu anys, i realitzar el mateix anàlisis.


## 2 Descripció de les dades
Els conjunts de dades en format csv són els següents:

1. Accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona
    - Llistat dels accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona. Incorpora el número de lesionats segons gravetat, el número de vehicles implicats i el punt d'impacte.
2. Vehicles implicats en accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona
    - Llistat dels vehicles implicats en accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona. Un accident pot tenir associat més d’un vehicle. Inclou si la causa és del vianant, el tipus de vehicle, model, marca, color i tipus carnet i antiguitat de la persona que ho conduïa.
3. Persones involucrades en accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona
    - Llistat de les persones que han estat involucrades en un accident gestionats per la Guàrdia Urbana a la ciutat de Barcelona i que han sofert algun tipus de lesió ( ferit lleu, ferit greu o mort). Inclou descripció de la persona (conductor, passatger o vianant), sexe, edat, vehicle associat a la persona i si la causa ha sigut del vianant.
4. Accidents segons tipologia gestionats per la Guàrdia Urbana a la ciutat de Barcelona
    - Relació d'accidents segons tipologia gestionats per la Guàrdia Urbana a la ciutat de Barcelona. Un accident pot estar tipificat amb vàries tipologies ( Ex: xoc contra element estàtic i col·lisió front-lateral)
5. Accidents segons causa mediata gestionats per la Guàrdia Urbana a la ciutat de Barcelona
    - Relació dels accidents segons causa mediata gestionats per la Guàrdia Urbana a la ciutat de Barcelona. Un accident pot tenir una o més causes mediates les quals fan referència a factors externs del resultat en temps, lloc o grau (Ex: Alcoholèmia i Excés de velocitat o velocitat inadequada)
6. Accidents segons causa conductor gestionats per la Guàrdia Urbana a la ciutat de Barcelona
    - Accidents segons causa conductor gestionats per la Guàrdia Urbana a la ciutat de Barcelona.


## 3. Càrrega de les dades i preparació de les dades per a l'anàlisi
Fem correr Spark en mode local, on tot el processament tindrà lloc en una sola màquina. En cas de necessitar instalar Spark, hi ha dos opcions: Anar a la [pàgina de descàrrega d'Spark](https://spark.apache.org/downloads.html), i triar "Pre-built for Apache Hadoop 3.3 and later", o bé, instalar la llibreria PySpark de Python i les seves dependencies invocant `pip install pyspark[sql, ml, mllib]`. A banda d'això, també necessitem tenir instalada la versió 8 o posterior de Java al nostre sistema.

In [50]:
# El proposit de findspark és facilitar l'ús de Spark amb Python,
# especialment si no tens declarada la variable d'entorn SPARK_HOME o
# si Spark i PySpark setup no es troben a la variable PATH.
# descomenta les següents dos linies si NO has instalar pyspark amb pip
#import findspark
#findspark.init()

from pyspark.sql import SparkSession
# inicialitzem amb 4 cores destinats a Spark (això dependrà de cada màquina)
spark = SparkSession.builder \
    .appName("bcn_traffic_incidents") \
    .master("local[4]") \
    .getOrCreate()

In [49]:
# si cal aturar-ho, executa spark.stop()
# spark.stop()

### 3.1 càrrega dels datasets com a DataFrames i creació de vistes temporals
Carreguem cada conjunt de dades en els següents DataFrames:

1. df_gu_incidents
2. df_gu_incidents_type
3. df_gu_incidents_vehicle
4. df_gu_incidents_person
5. df_gu_incidents_cause
6. df_gu_incidents_driver

Alhora registrarem cada DataFrame com una vista temporal (temporary view), el que ens permetrà executar comandes SQL directament en cadascuna d'elles.

Vistes temporals:

- incidents
- incidents_type
- incidents_vehicle
- incidents_person
- incidents_cause
- incidents_driver

  

In [51]:
incidents_file_path = "../data/2023_accidents_gu_bcn.csv"
incidents_type_file_path = "../data/2023_accidents_tipus_gu_bcn.csv"
incidents_vehicle_file_path = "../data/2023_accidents_vehicles_gu_bcn.csv"
incidents_person_file_path = "../data/2023_accidents_persones_gu_bcn.csv"
incidents_cause_file_path = "../data/2023_accidents_causes_gu_bcn.csv"
incidentsdriver_file_path = "../data/2023_accidents_causa_conductor_gu_bcn.csv"

df_gu_incidents = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .option("mode", "FAILFAST") # exit if any error
    .load(incidents_file_path))
df_gu_incidents.createOrReplaceTempView("incidents")

df_gu_incidents_type = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .option("mode", "FAILFAST") # exit if any error
    .load(incidents_type_file_path))
df_gu_incidents.createOrReplaceTempView("incidents_type")

df_gu_incidents_vehicle = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .option("mode", "FAILFAST") # exit if any error
    .load(incidents_vehicle_file_path))
df_gu_incidents.createOrReplaceTempView("incidents_vehicle")

df_gu_incidents_person = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .option("mode", "FAILFAST") # exit if any error
    .load(incidents_person_file_path))
df_gu_incidents.createOrReplaceTempView("incidents_person")

df_gu_incidents_cause = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .option("mode", "FAILFAST") # exit if any error
    .load(incidents_cause_file_path))
df_gu_incidents.createOrReplaceTempView("incidents_cause")

df_gu_incidents_driver = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .option("mode", "FAILFAST") # exit if any error
    .load(incidentsdriver_file_path))
df_gu_incidents.createOrReplaceTempView("incidents_driver")


### 3.2 Estructura de les dades

1. **df_gu_incidents**, accidents gestionats per la Guàrdia Urbana a Barcelona
2. **df_gu_incidents_type**, accidents segons tipologia
3. **df_gu_incidents_vehicle**, vehicles implicats en accidents
4. **df_gu_incidents_person**, persones involucrades en accidents
5. **df_gu_incidents_cause**, accidents segons causa mediata
6. **df_gu_incidents_driver**, accidents segons causa conductor

#### 3.2.1 df_gu_incidents

In [52]:
df_gu_incidents.printSchema()

root
 |-- Numero_expedient: string (nullable = true)
 |-- Codi_districte: integer (nullable = true)
 |-- Nom_districte: string (nullable = true)
 |-- Codi_barri: integer (nullable = true)
 |-- Nom_barri: string (nullable = true)
 |-- Codi_carrer: integer (nullable = true)
 |-- Nom_carrer: string (nullable = true)
 |-- Num_postal : string (nullable = true)
 |-- Descripcio_dia_setmana: string (nullable = true)
 |-- NK_Any: integer (nullable = true)
 |-- Mes_any: integer (nullable = true)
 |-- Nom_mes: string (nullable = true)
 |-- Dia_mes: integer (nullable = true)
 |-- Hora_dia: integer (nullable = true)
 |-- Descripcio_torn: string (nullable = true)
 |-- Descripcio_causa_vianant: string (nullable = true)
 |-- Numero_morts: integer (nullable = true)
 |-- Numero_lesionats_lleus: integer (nullable = true)
 |-- Numero_lesionats_greus: integer (nullable = true)
 |-- Numero_victimes: integer (nullable = true)
 |-- Numero_vehicles_implicats: integer (nullable = true)
 |-- Coordenada_UTM_Y_ED5

In [53]:
# mostrem les primeres 5 files
df_gu_incidents.show(5)

+----------------+--------------+--------------+----------+-----------------+-----------+--------------------+-----------+----------------------+------+-------+-------+-------+--------+---------------+------------------------+------------+----------------------+----------------------+---------------+-------------------------+---------------------+---------------------+--------------+-------------+
|Numero_expedient|Codi_districte| Nom_districte|Codi_barri|        Nom_barri|Codi_carrer|          Nom_carrer|Num_postal |Descripcio_dia_setmana|NK_Any|Mes_any|Nom_mes|Dia_mes|Hora_dia|Descripcio_torn|Descripcio_causa_vianant|Numero_morts|Numero_lesionats_lleus|Numero_lesionats_greus|Numero_victimes|Numero_vehicles_implicats|Coordenada_UTM_Y_ED50|Coordenada_UTM_X_ED50|Longitud_WGS84|Latitud_WGS84|
+----------------+--------------+--------------+----------+-----------------+-----------+--------------------+-----------+----------------------+------+-------+-------+-------+--------+-------------

In [54]:
# i el nombre total de files (accidents reportats per la guardia urbana)
df_gu_incidents.count()

7724

In [55]:
# mostrem el nombre de columnes
len(df_gu_incidents.columns)

25

Podem veure que en aquest conjunt de dades, corresponent a 2023, hi ha 7.724 accidents reportats, i cada accident consta de 25 característiques. Inicialment hem carregat les dades inferint directament l'esquema de dades, trobant inicialment que aquestes són de tipus `string` i `integer`. Veiem que en alguns casos, per exemple les coordenades WGS84, seria mes encertat que el tipus fos `float`.

De les característiques de les dades, les que considerem més interessants per al nostre treball, són les següents:

- Numero_expedient, considerem inicialment que aquesta actúa com a ID de l'accident, i ens permetrà vincular les dades entre els diferents datasets.
- Nom_districte
- Nom_barri
- Nom_carrer, interessant que en aquesta columna ens indica també els encreuaments de carrer on hi ha hagut l'incident, com per exemple "Badal / Càceres"
- Descripcio_dia_setmana
- NK_Any
- Mes_any, el mes en format numèric
- Dia_mes
- Hora_dia
- Descripcio_causa_vianant
- Numero_morts
- Numero_lesionats_lleus
- Numero_lesionats_greus, el dataset també aporta Numero_victimes, però aquesta la podriem inferir sumant número de morts i lesionats greus i lleus.
- Numero_vehicles_implicats
- Longitud_WGS84
- Latitud_WGS84, el dataset també ens aporta les dades de geolocalització UTM, però a efectes pràctics serà millor WGS84. Aquestes dades ens permetran localitzar els incidents quan han tingut lloc per exemple a les rondes.

Ens quedem amb 16 característiques de les 25 inicials.


#### 3.2.2 `df_gu_incidents_type`, accidents segons tipologia

In [56]:
df_gu_incidents_type.printSchema()
df_gu_incidents_type.count()

root
 |-- Numero_expedient: string (nullable = true)
 |-- Codi_districte: integer (nullable = true)
 |-- Nom_districte: string (nullable = true)
 |-- Codi_barri: integer (nullable = true)
 |-- Nom_barri: string (nullable = true)
 |-- Codi_carrer: integer (nullable = true)
 |-- Nom_carrer: string (nullable = true)
 |-- Num_postal: string (nullable = true)
 |-- Descripcio_dia_setmana: string (nullable = true)
 |-- NK_Any: integer (nullable = true)
 |-- Mes_any: integer (nullable = true)
 |-- Nom_mes: string (nullable = true)
 |-- Dia_mes: integer (nullable = true)
 |-- Hora_dia: integer (nullable = true)
 |-- Descripcio_torn: string (nullable = true)
 |-- Descripcio_tipus_accident: string (nullable = true)
 |-- Coordenada_UTM_X_ED50: string (nullable = true)
 |-- Coordenada_UTM_Y_ED50: string (nullable = true)
 |-- Longitud_WGS84: double (nullable = true)
 |-- Latitud_WGS84: double (nullable = true)



7823

In [57]:
len(df_gu_incidents_type.columns)

20

In [58]:
df_gu_incidents_type.show(5)

+----------------+--------------+-------------+----------+----------+-----------+--------------------+----------+----------------------+------+-------+--------+-------+--------+---------------+-------------------------+---------------------+---------------------+--------------+-------------+
|Numero_expedient|Codi_districte|Nom_districte|Codi_barri| Nom_barri|Codi_carrer|          Nom_carrer|Num_postal|Descripcio_dia_setmana|NK_Any|Mes_any| Nom_mes|Dia_mes|Hora_dia|Descripcio_torn|Descripcio_tipus_accident|Coordenada_UTM_X_ED50|Coordenada_UTM_Y_ED50|Longitud_WGS84|Latitud_WGS84|
+----------------+--------------+-------------+----------+----------+-----------+--------------------+----------+----------------------+------+-------+--------+-------+--------+---------------+-------------------------+---------------------+---------------------+--------------+-------------+
| 2023S004127    |            -1|   Desconegut|        -1|Desconegut|         -1|Diagonal         ...|3-35      |        

Veiem que conté 19 característiques presents en `df_gu_incidents` i una de nova: `Descripcio_tipus_accident`. Ens crida l'atenció que aquest dataset conté 7.823 accidents, 99 més que `df_gu_incidents`

#### 3.2.3 `df_gu_incidents_vehicle`, vehicles implicats en accidents

In [59]:
df_gu_incidents_vehicle.printSchema()
print(f"dataset conté {df_gu_incidents_vehicle.count()} accidents i {len(df_gu_incidents_vehicle.columns)} característiques")

root
 |-- Numero_expedient: string (nullable = true)
 |-- Codi_districte: integer (nullable = true)
 |-- Nom_districte: string (nullable = true)
 |-- Codi_barri: integer (nullable = true)
 |-- Nom_barri: string (nullable = true)
 |-- Codi_carrer: integer (nullable = true)
 |-- Nom_carrer: string (nullable = true)
 |-- Num_postal: string (nullable = true)
 |-- Descripcio_dia_setmana: string (nullable = true)
 |-- NK_Any: integer (nullable = true)
 |-- Mes_any: integer (nullable = true)
 |-- Nom_mes: string (nullable = true)
 |-- Dia_mes: integer (nullable = true)
 |-- Hora_dia: integer (nullable = true)
 |-- Descripcio_torn: string (nullable = true)
 |-- Descripcio_causa_vianant: string (nullable = true)
 |-- Descripcio_tipus_vehicle: string (nullable = true)
 |-- Descripcio_model: string (nullable = true)
 |-- Descripcio_marca: string (nullable = true)
 |-- Descripcio_color: string (nullable = true)
 |-- Descripcio_carnet: string (nullable = true)
 |-- Antiguitat_carnet: string (nullab

Aquest dataset recull les següents noves característiques:
- `Descripcio_tipus_vehicle`
- `Descripcio_model`
- `Descripcio_marca`
- `Descripcio_color`
- `Descripcio_carnet`
- `Antiguitat_carnet`

i recull 14.361 accidents durant 2023, 6.538 més que l'anterior dataset.

#### 3.2.4 `df_gu_incidents_person`, persones involucrades en accidents 

In [60]:
df_gu_incidents_person.printSchema()
print(f"dataset conté {df_gu_incidents_person.count()} accidents i {len(df_gu_incidents_person.columns)} característiques")

root
 |-- Numero_expedient: string (nullable = true)
 |-- Codi_districte: integer (nullable = true)
 |-- Nom_districte: string (nullable = true)
 |-- Codi_barri: integer (nullable = true)
 |-- Nom_barri: string (nullable = true)
 |-- Codi_carrer: integer (nullable = true)
 |-- Nom_carrer: string (nullable = true)
 |-- Num_postal: string (nullable = true)
 |-- Descripcio_dia_setmana: string (nullable = true)
 |-- NK_ Any: integer (nullable = true)
 |-- Mes_ any: integer (nullable = true)
 |-- Nom_mes: string (nullable = true)
 |-- Dia_mes: integer (nullable = true)
 |-- Descripcio_torn: string (nullable = true)
 |-- Hora_dia: integer (nullable = true)
 |-- Descripcio_causa_vianant: string (nullable = true)
 |-- Desc_Tipus_vehicle_implicat: string (nullable = true)
 |-- Descripcio_sexe: string (nullable = true)
 |-- Edat: string (nullable = true)
 |-- Descripció_tipus_persona: string (nullable = true)
 |-- Descripcio_Lloc_atropellament_vianant: string (nullable = true)
 |-- Descripcio_Mo

Veiem que les característiques úniques d'aquest dataset són:
- `Desc_Tipus_vehicle_implicat`
- `Descripcio_sexe`
- `Edat`
- `Descripció_tipus_persona`
- `Descripcio_Lloc_atropellament_vianant`
- `Descripcio_Motiu_desplacament_vianant`
- `Descripcio_Motiu_desplacament_conductor`
- `Descripcio_victimitzacio`

i recull 8.776 accidents.

#### 3.2.5 `df_gu_incidents_cause`, accidents segons causa mediata

In [61]:
df_gu_incidents_cause.printSchema()
print(f"dataset conté {df_gu_incidents_cause.count()} accidents i {len(df_gu_incidents_cause.columns)} característiques")

root
 |-- Numero_expedient: string (nullable = true)
 |-- Codi_districte: integer (nullable = true)
 |-- Nom_districte: string (nullable = true)
 |-- Codi_barri: integer (nullable = true)
 |-- Nom_barri: string (nullable = true)
 |-- Codi_carrer: integer (nullable = true)
 |-- Nom_carrer: string (nullable = true)
 |-- Num_postal : string (nullable = true)
 |-- Descripcio_dia_setmana: string (nullable = true)
 |-- NK_Any: integer (nullable = true)
 |-- Mes_any: integer (nullable = true)
 |-- Nom_mes: string (nullable = true)
 |-- Dia_mes: integer (nullable = true)
 |-- Hora_dia: integer (nullable = true)
 |-- Descripcio_torn: string (nullable = true)
 |-- Descripcio_causa_mediata: string (nullable = true)
 |-- Coordenada_UTM_X_ED50: string (nullable = true)
 |-- Coordenada_UTM_Y_ED50: string (nullable = true)
 |-- Longitud_WGS84: double (nullable = true)
 |-- Latitud_WGS84: double (nullable = true)

dataset conté 7721 accidents i 20 característiques


Conté una característica única: `Descripcio_causa_mediata`, veurem tot seguit quines són les causes més repetides:

In [62]:
causes = (df_gu_incidents_cause.select("Descripcio_causa_mediata")
          .rdd.map(lambda row: row["Descripcio_causa_mediata"])
          .map(lambda x: (x, 1))
          .reduceByKey(lambda a,b: a+b))
causes.take(20)

[('No hi ha causa mediata', 7208),
 ('Estat de la senyalització', 3),
 ('Objectes o animals a la calçada', 20),
 ('Calçada en mal estat', 53),
 ('Alcoholèmia', 396),
 ('Factors meteorològics', 6),
 ('Excés de velocitat o inadequada', 25),
 ('Drogues o medicaments', 10)]

#### 3.2.6 `df_gu_incidents_driver`, accidents segons causa conductor

In [63]:
df_gu_incidents_driver.printSchema()
print(f"dataset conté {df_gu_incidents_driver.count()} accidents i {len(df_gu_incidents_driver.columns)} característiques")

root
 |-- Numero_expedient: string (nullable = true)
 |-- Codi_districte: integer (nullable = true)
 |-- Nom_districte: string (nullable = true)
 |-- Codi_barri: integer (nullable = true)
 |-- Nom_barri: string (nullable = true)
 |-- Codi_carrer: integer (nullable = true)
 |-- Nom_carrer: string (nullable = true)
 |-- Num_postal : string (nullable = true)
 |-- Descripcio_dia_setmana: string (nullable = true)
 |-- NK_Any: integer (nullable = true)
 |-- Mes_any: integer (nullable = true)
 |-- Nom_mes: string (nullable = true)
 |-- Dia_mes: integer (nullable = true)
 |-- Hora_dia: integer (nullable = true)
 |-- Descripcio_torn: string (nullable = true)
 |-- Descripcio_causa_mediata: string (nullable = true)
 |-- Coordenada_UTM_X_ED50: double (nullable = true)
 |-- Coordenada_UTM_Y_ED50: double (nullable = true)
 |-- Longitud_WGS84: double (nullable = true)
 |-- Latitud_WGS84: double (nullable = true)

dataset conté 8897 accidents i 20 característiques


Aquest conjunt de dades conté les mateixes característiques que l'anterior `df_gu_incidents_cause` i recull 8.897 accidents, 1.176 més que l'anterior. Podem veure que en aquest dataset `Descripcio_causa_mediata` recull la causa de l'accident de la qual n'es responsable el conductor:

In [64]:
driver_causes = (df_gu_incidents_driver.select("Descripcio_causa_mediata")
          .rdd.map(lambda row: row["Descripcio_causa_mediata"])
          .map(lambda x: (x, 1))
          .reduceByKey(lambda a,b: a+b))
driver_causes.take(20)

[("Manca d'atenció a la conducció", 2061),
 ('No respectar distàncies', 996),
 ('No respectat pas de vianants', 241),
 ('Desobeir semàfor', 640),
 ('Desobeir altres senyals', 457),
 ('Gir indegut o sense precaució', 804),
 ('Altres', 1089),
 ('Desconegut', 377),
 ('Avançament defectuós/improcedent', 296),
 ('Canvi de carril sense precaució', 666),
 ('No determinada', 743),
 ('Manca precaució incorporació circulació', 238),
 ('No cedir la dreta', 72),
 ('Manca precaució efectuar marxa enrera', 177),
 ('Envair calçada contrària', 25),
 ('Fallada mecànica o avaria', 15)]

In [65]:
#spark.sql(
#    """
#    SELECT Nom_carrer, Dia_mes, Numero_morts, Numero_vehicles_implicats, Descripcio_causa_vianant
#    FROM incidents WHERE Numero_morts > 0
#    ORDER BY Numero_morts DESC
#    """
#).show(10)

### 3.3 Preparació de les dades
Crearem un nou data frame de cada conjunt de dades amb les característiques seleccionades, els hi donem un nom més concis.

#### 3.3.1 df_gu_incidents -> df_incidents

Descartem les següents característiques del conjunt de dades inicial:

- Codi_districte
- Codi_barri
- Codi_carrer
- Num_postal
- Nom_mes
- Descripcio_torn
- Numero_victimes
- Coordenada_UTM_Y_ED50
- Coordenada_UTM_X_ED50

i conservem les següents, a les que els hi donarem un nom mes curt:

- Numero_expedient : expedient
- Nom_districte : districte
- Nom_barri : barri
- Nom_carrer : carrer
- Descripcio_dia_setmana : dia_nom
- NK_Any : any
- Mes_any : mes
- Dia_mes : dia
- Hora_dia: hora
- Descripcio_causa_vianant: causa_vianant
- Numero_morts: morts
- Numero_lesionats_lleus: lleus
- Numero_lesionats_greus: greus
- Numero_vehicles_implicats: vehicles
- Longitud_WGS84: long
- Latitud_WGS84: lat


In [79]:
# We could do the same query with the DataFram API
from pyspark.sql.functions import col, desc
#(df_gu_incidents.select(df_gu_incidents["Nom_carrer"].alias("carrer"), "Dia_mes", "Numero_morts", "Numero_vehicles_implicats", "Descripcio_causa_vianant", "Num_postal ")
#     .where(col("Num_postal ") IS NULL)
#     .orderBy("Numero_morts", ascending=False).show(10))

column_mapping = {
        "Numero_expedient": "expedient", 
        "Nom_districte": "districte",
        "Nom_barri": "barri",
        "Nom_carrer": "carrer",
        "Descripcio_dia_setmana": "dia_nom",
        "NK_Any": "any", 
        "Mes_any": "mes", 
        "Dia_mes": "dia",
        "Hora_dia": "hora", 
        "Descripcio_causa_vianant": "causa_vianant", 
        "Numero_morts": "morts", 
        "Numero_lesionats_lleus": "lleus", 
        "Numero_lesionats_greus": "greus",
        "Numero_vehicles_implicats": "num_vehicles",
        "Longitud_WGS84": "long",
        "Latitud_WGS84": "lat"    
}

features = [x for x,y in column_mapping.items()]

df_incidents = df_gu_incidents.select(features)

for old_col, new_col in column_mapping.items():
    df_incidents = df_incidents.withColumnRenamed(old_col, new_col)

df_incidents.show(5)

+---------------+--------------+-----------------+--------------------+--------+----+---+---+----+--------------------+-----+-----+-----+------------+-----------+------------+
|      expedient|     districte|            barri|              carrer| dia_nom| any|mes|dia|hora|       causa_vianant|morts|lleus|greus|num_vehicles|       long|         lat|
+---------------+--------------+-----------------+--------------------+--------+----+---+---+----+--------------------+-----+-----+-----+------------+-----------+------------+
|2023S000001    |Sants-Montjuïc|            Sants|Badal / Càceres  ...|Diumenge|2023|  1|  1|  10|No és causa del  ...|    0|    0|    1|           1|2.130813369|41.374565193|
|2023S000002    |Horta-Guinardó| el Baix Guinardó|Encarnació       ...|Diumenge|2023|  1|  1|   3|No és causa del  ...|    0|    0|    0|           1|2.165512751|41.409867042|
|2023S000003    |  Ciutat Vella|         el Raval|Carme            ...|Diumenge|2023|  1|  1|   3|Creuar per fora p...| 

#### 3.3.2 df_gu_incidents_type -> df_incidents_type

conservem les següents, a les que els hi donarem un nom mes curt:

- Numero_expedient : expedient
- Nom_districte : districte
- Nom_barri : barri
- Nom_carrer : carrer
- Descripcio_dia_setmana : dia_nom
- NK_Any : any
- Mes_any : mes
- Dia_mes : dia
- Hora_dia: hora
- Descripcio_tipus_accident: tipus_accident
- Longitud_WGS84: long
- Latitud_WGS84: lat


In [78]:
column_mapping = {
        "Numero_expedient": "expedient", 
        "Nom_districte": "districte",
        "Nom_barri": "barri",
        "Nom_carrer": "carrer",
        "Descripcio_dia_setmana": "dia_nom",
        "NK_Any": "any", 
        "Mes_any": "mes", 
        "Dia_mes": "dia",
        "Hora_dia": "hora", 
        "Descripcio_tipus_accident": "tipus_accident", 
        "Longitud_WGS84": "long",
        "Latitud_WGS84": "lat"    
}

features = [x for x,y in column_mapping.items()]

df_incidents_type = df_gu_incidents_type.select(features)

for old_col, new_col in column_mapping.items():
    df_incidents_type = df_incidents_type.withColumnRenamed(old_col, new_col)

df_incidents_type.show(5)

+---------------+----------+----------+--------------------+--------+----+---+---+----+-----------------+-----------+------------+
|      expedient| districte|     barri|              carrer| dia_nom| any|mes|dia|hora|   tipus_accident|       long|         lat|
+---------------+----------+----------+--------------------+--------+----+---+---+----+-----------------+-----------+------------+
|2023S004127    |Desconegut|Desconegut|Diagonal         ...|Dissabte|2023|  7|  8|   5|            Abast|2.216403377|41.410440789|
|2023S003871    |Desconegut|Desconegut|Colom / Portal de...| Dimarts|2023|  6| 27|  19|            Abast|2.178020316|41.376109822|
|2023S005242    |Desconegut|Desconegut|Numància         ...| Dilluns|2023|  9|  4|  12|Col.lisió lateral|2.141503645|41.381380404|
|2023S005352    |Desconegut|Desconegut|Aigües           ...|Dissabte|2023|  9|  9|  16|       Desconegut|2.134478091|41.421926017|
|2023S007652    |Desconegut|Desconegut|Diagonal         ...|Diumenge|2023| 12| 24| 

### 3.3.3 df_gu_incidents_vehicle -> df_incidents_vehicle

conservem les següents, a les que els hi donarem un nom mes curt:

- Numero_expedient : expedient
- Nom_districte : districte
- Nom_barri : barri
- Nom_carrer : carrer
- Descripcio_dia_setmana : dia_nom
- NK_Any : any
- Mes_any : mes
- Dia_mes : dia
- Hora_dia: hora
- Descripcio_causa_vianant: causa_vianant
- Descripcio_tipus_vehicle: tipus_vehicle
- Descripcio_carnet: carnet
- Antiguitat_carnet: carnet_anys
- Longitud_WGS84: long
- Latitud_WGS84: lat


In [77]:
column_mapping = {
        "Numero_expedient": "expedient", 
        "Nom_districte": "districte",
        "Nom_barri": "barri",
        "Nom_carrer": "carrer",
        "Descripcio_dia_setmana": "dia_nom",
        "NK_Any": "any", 
        "Mes_any": "mes", 
        "Dia_mes": "dia",
        "Hora_dia": "hora", 
        "Descripcio_causa_vianant": "causa_vianant",
        "Descripcio_tipus_vehicle": "tipus_vehicle",
        "Descripcio_carnet": "carnet",
        "Antiguitat_carnet": "carnet_anys",
        "Longitud_WGS84": "long",
        "Latitud_WGS84": "lat"    
}

features = [x for x,y in column_mapping.items()]

df_incidents_vehicle = df_gu_incidents_vehicle.select(features)

for old_col, new_col in column_mapping.items():
    df_incidents_vehicle = df_incidents_vehicle.withColumnRenamed(old_col, new_col)

df_incidents_vehicle.show(5)

+---------------+----------+----------+--------------------+-------+----+---+---+----+--------------------+-----------------+----------+-----------+-----------+------------+
|      expedient| districte|     barri|              carrer|dia_nom| any|mes|dia|hora|       causa_vianant|    tipus_vehicle|    carnet|carnet_anys|       long|         lat|
+---------------+----------+----------+--------------------+-------+----+---+---+----+--------------------+-----------------+----------+-----------+-----------+------------+
|2023S000536    |Desconegut|Desconegut|Motors / Acer    ...| Dijous|2023|  1| 26|  14|No és causa del  ...|          Turisme|Desconegut| Desconegut|2.139637367|41.348935697|
|2023S000506    |Desconegut|Desconegut|Motors           ...| Dijous|2023|  1| 26|  15|No és causa del  ...|      Motocicleta|         B|         11|2.148801757|41.352030518|
|2023S000506    |Desconegut|Desconegut|                NULL| Dijous|2023|  1| 26|  15|No és causa del  ...|          Turisme|     

### 3.3.4 df_gu_incidents_person -> df_incidents_person

conservem les següents, a les que els hi donarem un nom mes curt:

- Numero_expedient : expedient
- Nom_districte : districte
- Nom_barri : barri
- Nom_carrer : carrer
- Descripcio_dia_setmana : dia_nom
- NK_Any : any
- Mes_any : mes
- Dia_mes : dia
- Hora_dia: hora
- Descripcio_causa_vianant: causa_vianant
- Desc_Tipus_vehicle_implicat: tipus_vehicle
- Descripcio_sexe: sexe
- Edat: edat
- Descripció_tipus_persona: tipus_persona
- Descripcio_Lloc_atropellament_vianant: lloc
- Descripcio_Motiu_desplacament_vianant: motiu_desp_vianant
- Descripcio_Motiu_desplacament_conductor: motiu_desp_conductor
- Descripcio_victimitzacio: victimitzacio
- Longitud_WGS84: long
- Latitud_WGS84: lat


In [83]:
column_mapping = {
        "Numero_expedient": "expedient", 
        "Nom_districte": "districte",
        "Nom_barri": "barri",
        "Nom_carrer": "carrer",
        "Descripcio_dia_setmana": "dia_nom",
        "NK_ Any": "any", 
        "Mes_ any": "mes", 
        "Dia_mes": "dia",
        "Hora_dia": "hora", 
        "Descripcio_causa_vianant": "causa_vianant",
        "Desc_Tipus_vehicle_implicat": "tipus_vehicle",
        "Descripcio_sexe": "sexe",
        "Edat": "edat",
        "Descripció_tipus_persona": "tipus_persona",
        "Descripcio_Lloc_atropellament_vianant": "lloc",
        "Descripcio_Motiu_desplacament_vianant": "motiu_desp_vianant",
        "Descripcio_Motiu_desplacament_conductor": "motiu_desp_conductor",
        "Descripcio_victimitzacio": "victimitzacio",
        "Longitud_WGS84": "long",
        "Latitud_WGS84": "lat"    
}

features = [x for x,y in column_mapping.items()]

df_incidents_person = df_gu_incidents_person.select(features)

for old_col, new_col in column_mapping.items():
    df_incidents_person = df_incidents_person.withColumnRenamed(old_col, new_col)

df_incidents_person.show(5)

+---------------+--------------+--------------------+--------------------+--------+----+---+---+----+--------------------+--------------------+----+----+-------------+--------------------+------------------+--------------------+--------------------+-----------+------------+
|      expedient|     districte|               barri|              carrer| dia_nom| any|mes|dia|hora|       causa_vianant|       tipus_vehicle|sexe|edat|tipus_persona|                lloc|motiu_desp_vianant|motiu_desp_conductor|       victimitzacio|       long|         lat|
+---------------+--------------+--------------------+--------------------+--------+----+---+---+----+--------------------+--------------------+----+----+-------------+--------------------+------------------+--------------------+--------------------+-----------+------------+
|2023S000001    |Sants-Montjuïc|               Sants|               Badal|Diumenge|2023|  1|  1|  10|No és causa del  ...|Camió rígid > 3,5...|Home|  57|      Vianant|En zona 

In [21]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/Users/pau/coding-projects/ita-barcelona-traffic-incidents/notebook/spark-warehouse')]

In [25]:
#spark.catalog.listTables()

In [24]:
#spark.catalog.listColumns("incidents")

#### Causes according the pedestrian - Causes a causa del vianant

Anàlisis de les principals causes a causa del vianant.
Creem un llistat amb les causes i el nombre de vegades

In [9]:
causes = (df_gu_incidents.select("Descripcio_causa_vianant")
          .rdd.map(lambda row: row["Descripcio_causa_vianant"])
          .map(lambda x: (x, 1))
          .reduceByKey(lambda a,b: a+b))

In [10]:
causes.take(20)

[('No és causa del  vianant', 7249),
 ('Creuar per fora pas de vianants', 147),
 ('Altres', 92),
 ('Desobeir el senyal del semàfor', 181),
 ('Transitar a peu per la calçada', 48),
 ('Desobeir altres senyals', 7)]

Podem aconseguir el mateix fent servir DataFrame API enlloc de RDD

In [11]:
causes = df_gu_incidents.groupBy("Descripcio_causa_vianant").count()

In [12]:
causes.orderBy("count", ascending=False).show(truncate=False)

+-------------------------------+-----+
|Descripcio_causa_vianant       |count|
+-------------------------------+-----+
|No és causa del  vianant       |7249 |
|Desobeir el senyal del semàfor |181  |
|Creuar per fora pas de vianants|147  |
|Altres                         |92   |
|Transitar a peu per la calçada |48   |
|Desobeir altres senyals        |7    |
+-------------------------------+-----+



In [13]:
causes_list = [(row['Descripcio_causa_vianant'], row['count']) for row in causes.orderBy("count", ascending=False).collect()]
causes_list

[('No és causa del  vianant', 7249),
 ('Desobeir el senyal del semàfor', 181),
 ('Creuar per fora pas de vianants', 147),
 ('Altres', 92),
 ('Transitar a peu per la calçada', 48),
 ('Desobeir altres senyals', 7)]

In [14]:
percent_no_causa_vianant = 100 * causes_list[0][1] / df_gu_incidents.count()

In [15]:
percent_no_causa_vianant

93.8503366131538

El 93.85% dels accidents de l'any 2023 no són causa del vianant. De la resta que si ho són, la principal causa ha estat desobeïr el senyal del semàfor, seguida per creuar fora del pas de vianants.

### df_gu_incidents_type

In [16]:
df_gu_incidents_type.show(5, truncate=False)
df_gu_incidents_type.count()

+----------------+--------------+-------------+----------+----------+-----------+--------------------------------------------------+----------+----------------------+------+-------+--------+-------+--------+---------------+-------------------------+---------------------+---------------------+--------------+-------------+
|Numero_expedient|Codi_districte|Nom_districte|Codi_barri|Nom_barri |Codi_carrer|Nom_carrer                                        |Num_postal|Descripcio_dia_setmana|NK_Any|Mes_any|Nom_mes |Dia_mes|Hora_dia|Descripcio_torn|Descripcio_tipus_accident|Coordenada_UTM_X_ED50|Coordenada_UTM_Y_ED50|Longitud_WGS84|Latitud_WGS84|
+----------------+--------------+-------------+----------+----------+-----------+--------------------------------------------------+----------+----------------------+------+-------+--------+-------+--------+---------------+-------------------------+---------------------+---------------------+--------------+-------------+
|2023S004127     |-1           

7823

In [18]:
causes_type = df_gu_incidents_type.groupBy("Descripcio_tipus_accident").count()
causes_type.orderBy("count", ascending=False).show(truncate=False)

+----------------------------------+-----+
|Descripcio_tipus_accident         |count|
+----------------------------------+-----+
|Col.lisió lateral                 |1812 |
|Abast                             |1638 |
|Col.lisió fronto-lateral          |1234 |
|Atropellament                     |927  |
|Xoc contra element estàtic        |653  |
|Caiguda (dues rodes)              |580  |
|Caiguda interior vehicle          |367  |
|Abast multiple                    |209  |
|Altres                            |181  |
|Col.lisió frontal                 |143  |
|Encalç                            |26   |
|Desconegut                        |19   |
|Xoc amb animal a la calçada       |13   |
|Sortida de via amb xoc o col.lisió|12   |
|Bolcada (més de dues rodes)       |7    |
|Sortida de via amb bolcada        |2    |
+----------------------------------+-----+



Les col·lisions per **abast** es produeixen si dos vehicles topen de manera que la part anterior de l’un impacta contra la part posterior de l’altre. Quan hi ha més de dos vehicles implicats es parla d’abast múltiple.

col·lisió per **encalç**: Col·lisió en què la part frontal d'un vehicle xoca amb la part posterior d'un altre vehicle.

Abast i encalç són el mateix.


In [20]:
causes_type_list = [(row['Descripcio_tipus_accident'], row['count']) for row in causes_type.orderBy("count", ascending=False).collect()]
causes_type_list

[('Col.lisió lateral', 1812),
 ('Abast', 1638),
 ('Col.lisió fronto-lateral', 1234),
 ('Atropellament', 927),
 ('Xoc contra element estàtic', 653),
 ('Caiguda (dues rodes)', 580),
 ('Caiguda interior vehicle', 367),
 ('Abast multiple', 209),
 ('Altres', 181),
 ('Col.lisió frontal', 143),
 ('Encalç', 26),
 ('Desconegut', 19),
 ('Xoc amb animal a la calçada', 13),
 ('Sortida de via amb xoc o col.lisió', 12),
 ('Bolcada (més de dues rodes)', 7),
 ('Sortida de via amb bolcada', 2)]

### df_gu_incidents_vehicle

In [21]:
df_gu_incidents_vehicle.show(5, truncate=False)
df_gu_incidents_vehicle.count()

+----------------+--------------+-------------+----------+----------+-----------+--------------------------------------------------+----------+----------------------+------+-------+-------+-------+--------+---------------+------------------------+------------------------+----------------+----------------+----------------+-----------------+-----------------+---------------------+---------------------+--------------+-------------+
|Numero_expedient|Codi_districte|Nom_districte|Codi_barri|Nom_barri |Codi_carrer|Nom_carrer                                        |Num_postal|Descripcio_dia_setmana|NK_Any|Mes_any|Nom_mes|Dia_mes|Hora_dia|Descripcio_torn|Descripcio_causa_vianant|Descripcio_tipus_vehicle|Descripcio_model|Descripcio_marca|Descripcio_color|Descripcio_carnet|Antiguitat_carnet|Coordenada_UTM_X_ED50|Coordenada_UTM_Y_ED50|Longitud_WGS84|Latitud_WGS84|
+----------------+--------------+-------------+----------+----------+-----------+--------------------------------------------------+--

24/04/30 18:48:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


14361

In [22]:
causes_vehicle_type = df_gu_incidents_vehicle.groupBy("Descripcio_causa_vianant").count()
causes_vehicle_type.orderBy("count", ascending=False).show(truncate=False)

+-------------------------------+-----+
|Descripcio_causa_vianant       |count|
+-------------------------------+-----+
|No és causa del  vianant       |13843|
|Desobeir el senyal del semàfor |192  |
|Creuar per fora pas de vianants|160  |
|Altres                         |102  |
|Transitar a peu per la calçada |52   |
|Desobeir altres senyals        |12   |
+-------------------------------+-----+



In [31]:
causes_vehicle_type = df_gu_incidents_vehicle.groupBy("Descripcio_tipus_vehicle").count()
causes_vehicle_type.orderBy("count", ascending=False).show(35, truncate=False)

+-----------------------------------+-----+
|Descripcio_tipus_vehicle           |count|
+-----------------------------------+-----+
|Turisme                            |5364 |
|Motocicleta                        |4340 |
|Furgoneta                          |984  |
|Veh. mobilitat personal amb motor  |772  |
|Bicicleta                          |724  |
|Autobús                            |476  |
|Taxi                               |402  |
|Ciclomotor                         |383  |
|Camió rígid <= 3,5 tones           |199  |
|Camió rígid > 3,5 tones            |163  |
|Autobús articulat                  |163  |
|Tot terreny                        |132  |
|Desconegut                         |56   |
|Tractor camió                      |49   |
|Ambulància                         |31   |
|Altres vehicles amb motor          |29   |
|Maquinària d'obres i serveis       |23   |
|Veh. mobilitat personal sense motor|18   |
|Tren o tramvia                     |14   |
|Tricicle                       

In [33]:
causes_vehicle_type = df_gu_incidents_vehicle.groupBy("Antiguitat_carnet").count()
causes_vehicle_type.orderBy("count", ascending=False).show(40, truncate=False)

+-----------------+-----+
|Antiguitat_carnet|count|
+-----------------+-----+
|Desconegut       |2873 |
|2                |549  |
|1                |543  |
|5                |445  |
|4                |419  |
|3                |384  |
|6                |379  |
|7                |378  |
|15               |368  |
|16               |335  |
|8                |320  |
|14               |318  |
|17               |315  |
|9                |313  |
|18               |309  |
|13               |304  |
|12               |290  |
|11               |285  |
|19               |284  |
|10               |282  |
|20               |265  |
|21               |222  |
|28               |215  |
|27               |209  |
|31               |205  |
|30               |203  |
|34               |197  |
|29               |194  |
|32               |194  |
|33               |186  |
|36               |186  |
|26               |170  |
|35               |166  |
|22               |165  |
|25               |158  |
|23         